In [111]:
import pandas as pd
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import random

In [112]:
df = pd.read_csv('war_and_peace_chapter1.txt', sep="\n", header=None)

In [113]:
def basic_clean(text):
    #wnl = nltk.stem.WordNetLemmatizer()
    text = (unicodedata.normalize('NFKD', text)
            .encode('ascii', 'ignore')
            .decode('utf-8', 'ignore')
            .lower())
    words = re.sub(r'[^\w\s. ]', '', text).split()
    return words
    #return [wnl.lemmatize(word) for word in words]

In [114]:
words = basic_clean(' '.join(df[0].tolist()))

In [115]:
one_grams = pd.Series(nltk.ngrams(words, 1)).value_counts()  # generating bi-gram

In [116]:
most_frequent_words=[]
for i in range(30):
    most_frequent_words.append(one_grams.index[i][0])

In [117]:
def gen_ngrams(length):
    ngrams = {}
    length = length

    for i in range(len(words)-length):
        seq = ' '.join(words[i:i+length])
        if seq not in ngrams.keys():
            ngrams[seq] = {}
        if words[i+length] not in ngrams[seq].keys():
            ngrams[seq][words[i+length]] = 1
        else:
            ngrams[seq][words[i+length]] = ngrams[seq][words[i+length]] + 1
    
    for i in ngrams.keys():
        ngrams[i] = {k: v for k, v in reversed(sorted(ngrams[i].items(), key=lambda item: item[1]))}
    
    return ngrams

In [118]:
two_grams = gen_ngrams(1)
three_grams = gen_ngrams(2)
four_grams = gen_ngrams(3)

In [119]:
def generate_next_word(ngrams, curr_sequence):
    high_prob_words = []
    keys = ngrams[curr_sequence].keys()
    highest_val = ngrams[curr_sequence][list(keys)[0]]
    for i in keys:
        val = ngrams[curr_sequence][i]
        if val < highest_val:
            break
        high_prob_words.append(i)
    return high_prob_words[random.randrange(len(high_prob_words))]  

In [120]:
def smoothing(ngrams, curr_sequence, length, output):
    
    if len(curr_sequence) < length:
        for i in range(length-len(curr_sequence)):
            if (nltk.word_tokenize(curr_sequence)[-1]) not in words:
                curr_sequence += ' ' + most_frequent_words[random.randrange(len(most_frequent_words))]
            else:
                curr_sequence += ' ' + generate_next_word(two_grams, curr_sequence)
        output = curr_sequence
    print(curr_sequence)
    while(curr_sequence not in ngrams.keys()):
        if (nltk.word_tokenize(curr_sequence)[-1]) not in words:
            next_word = most_frequent_words[random.randrange(len(most_frequent_words))]
            print(next_word)
        else:
            next_word = generate_next_word(two_grams, curr_sequence)
        output += ' ' + next_word
        seq_words = nltk.word_tokenize(output)
        curr_sequence = seq_words[-1]
        print(curr_sequence)

    return curr_sequence, output

In [121]:
def generate_sentence(ngrams, length, curr_sequence):
    output = curr_sequence
    for i in range(25):
        if curr_sequence not in ngrams.keys():
            next_word = most_frequent_words[random.randrange(len(most_frequent_words))]
        else:
            next_word = generate_next_word(ngrams, curr_sequence)
        output += ' ' + next_word
        seq_words = nltk.word_tokenize(output)
        curr_sequence = ' '.join(seq_words[len(seq_words)-length:len(seq_words)])

    print(output)

In [154]:
generate_sentence(four_grams, 3, 'and having got')

and having got said a not of that that you and not said to anna as to and for by and of not has was has had it


In [140]:
def generate_one_gram_sequence(output):
    for i in range(20):
        next_word = most_frequent_words[random.randrange(len(most_frequent_words))]
        output+=' ' + next_word
    return(output)

In [143]:
generate_one_gram_sequence('not two minutes')

'not two minutes had his had not with have in a she be anna that his not and her that not she of'